In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'
train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')

df = pd.concat([train, test])

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  import sys


In [ ]:
mmmmm = {'Secondary / secondary special': 2,
         'Higher education': 4,
         'Incomplete higher': 3,
         'Lower secondary': 1,
         'Academic degree': 5}

df['NAME_EDUCATION_TYPE'] = df['NAME_EDUCATION_TYPE'].map(mmmmm)

In [2]:
def count(df, gp_col):
    group = df.groupby(gp_col).size().reset_index().rename(index=str, columns={0:'count_' + '_'.join(gp_col)})
    return group

def unique(df, gp_col, col):
    group = df.groupby(gp_col)[[col]].nunique().reset_index().rename(index=str, columns={col:'unique_' + '_'.join(gp_col)})
    return group

In [3]:
import os
if not os.path.exists(PATH + 'inter/train_sup/'): os.mkdir(PATH + 'inter/train_sup/')

In [4]:
t = df.copy()

In [5]:
for x in [['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE'], ['CODE_GENDER', 'NAME_EDUCATION_TYPE'], ['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE'], ['CODE_GENDER', 'NAME_INCOME_TYPE']]:
    print(x)
    t = t.merge(count(df, x), on=x, how='left')
    
l = [
    ['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE'],
    ['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE', 'CODE_GENDER'],
    ['CODE_GENDER', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE'],
    ['CODE_GENDER', 'NAME_EDUCATION_TYPE'], ['NAME_EDUCATION_TYPE', 'CODE_GENDER'],
    ['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE'], ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE'],
    ['CODE_GENDER', 'NAME_INCOME_TYPE'], ['NAME_INCOME_TYPE', 'CODE_GENDER'],
]
for x in l:
    print(x)
    t = t.merge(unique(df, x[:-1], x[-1]), on=x[:-1], how='left')

['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']
['CODE_GENDER', 'NAME_EDUCATION_TYPE']
['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']
['CODE_GENDER', 'NAME_INCOME_TYPE']
['CODE_GENDER', 'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']
['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE', 'CODE_GENDER']
['CODE_GENDER', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE']
['CODE_GENDER', 'NAME_EDUCATION_TYPE']
['NAME_EDUCATION_TYPE', 'CODE_GENDER']
['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']
['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE']
['CODE_GENDER', 'NAME_INCOME_TYPE']
['NAME_INCOME_TYPE', 'CODE_GENDER']


In [7]:
t['MONTHS_BIRTH'] = (t['DAYS_BIRTH'] / 30).astype('int64')
t['CAT_MONTHS_BIRTH'] = (t['MONTHS_BIRTH'] % 12).astype('int8')
t['YEARS_BIRTH'] = (t['MONTHS_BIRTH'] / 12).astype('int64')
t['RATE_YEARS_BIRTH'] = t['YEARS_BIRTH'] / t['NAME_EDUCATION_TYPE']

In [10]:
tmp = t[[x for x in t.columns if x not in df.columns]]
tmp.shape

(356255, 15)

In [11]:
tmp.to_pickle(PATH + 'inter/train_sup/train_sup.pkl')